# **Interacting with databases with Python**

### **Overview**

- ***PEP 249***: The DB API 2.0 spesification
- ***Connections***: Managing your link to the database
- ***Cursors***: Running queries and getting the results back
- A word about ORMs

**PEP 249 defined the DB API 2.0, which is implemented by all Python database modules**

- PEP 249 (Python Enhancement Proposal) dates back to 2021
- Similar to pre-SQL database languages, there were no conventions for how to<br>
connect to databases with Python.
- PEP 249 specifies a common API which is used accross all modules that implement<br>
a database connection, across database types (relational, graph, document..).
- PEP 249 has two core object types: **connections** and **cursors**.
- We'll use examples from psycopg2, the Postgres modules for Python

---

### **Connections and cursors at a glance**

- **Connection**:<br> The "boringest" object which only deals with connection <br> to the database and closing the connection when done.
- **Cursor**:<br> The worker object which is spawned by the connection object.
    - Eats raw SQL!

In [ ]:
import psycopg2

# Connect to an existing database
conn = psycopg2.connect("dbname=test user=postgres")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: this creates a new table
cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")

# Pass data to fill a query placeholdersand let Psygopg perform 
# the correct conversion (no more SQL injections!)
cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (100, "abc'def"))

# Query the database and obtain data as Python objects
cur.execute("SELECT * FROM test;")
cur.fetchone()

# Make the changes to the database persistent
conn.commit()

# Close communication with the database
cur.close()
conn.close()

### **Connections**

1. They are opened.
2. They pass the cursor.
3. They are closed.

You are responsible for closing your own connections
- This applies especially to you, person who uses jupyter notebook to practice this.<br>
(Especially if you use a shared database, such as the one in Azure.

### **Cursors**

Cursors are THE interface to the database.

Cursors allow you to run SQL queries directly with the ***.execute()*** and ***.executemany()***
- It is strongly recomennded that parameters are passed as the second argument to these functions, <br>
rather than using strong concatenation. (remember SQL injection?)

Once a query has been executed, use the ***.fetchone()***, ***.fetchmany()***, and/or ***.fetchall()*** <br>
to retrieve rows from the return set as (a list of) Python tuples.

Cursors should also be closed when you're done with them.

**PEP 249 specifies that autocommit is of by default, so everything you do must be committed with <br> the .commit() method.**

Detail: All cursors spawned from the same connection object can "see" each others changes even <br> before transactions are committed!

Remember to NEVER commit data which would leave your database in an inconsistent state.

Use ***.rollback()*** to rollback the transaction in case of errors

---

### **A word about context managers in Python**

Do you hate forgetting to close cursors?

Some ephemeral objects like connections and cursors are commonly used with <br>
Python's **context manager** feature.

A context is created using the **with** keyword

If a cursor is created in a context header, it is closed when the scope is finished.

In [ ]:
with open("test.txt") as f:
    data = f.read()

# **EXAMPLES**

In [ ]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres password=mysecretpassword", port=5544)
cur = conn.cursor()

1. Create a table "oscar_winners" with the following schema using the psycopg2 package:

- id serial PRIMARY KEY<br>
- year integer<br>
- age integer<br>
- name varchar<br>
- movie varchar<br>


In [ ]:
def create_oscar_winners_table():
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS oscar_winners;")
        cur.execute("CREATE TABLE oscar_winners (id serial PRIMARY KEY, year integer, age integer, name varchar, movie varchar);")
        conn.commit()

create_oscar_winners_table()

2. Download the file "oscar_age_males.csv" from canvas and insert it to the oscar_winners table using DBeaver.<br> (You can do this by right clicking the table in DBeaver, and selecting import data, then select the file)

3. Create the function get_youngest_oscar_winner() that queries the database and returns the name, age and movie of the youngest oscar winner.

In [ ]:
def get_youngest_oscar_winner():
    with conn.cursor() as cur:
        cur.execute("SELECT name, age, movie from oscar_winners ORDER BY age LIMIT 1")
        return list(cur.fetchone())

get_youngest_oscar_winner()

4. Create the function get_winners_by_age(age) that has age as a parameter, queries the database for all <br> oscar winners that are that age and returns them as a list of lists.

In [ ]:
def get_winners_by_age(age):
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM oscar_winners WHERE age = %s", [age])
        # Since task asked for a list, convert each element of the list to a list
        return [list(x) for x in cur.fetchall()]

get_winners_by_age(37)

5. Create the function get_winners_by_name(name) that has name as a parameter and returns all oscar winners<br> in the database that has a name that contains the given name.

In [ ]:
def get_winners_by_name(name):
    name_formatted = f'%{name.lower()}%'
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM oscar_winners WHERE name like %s", [name_formatted])
        # Since task asked for a list, convert each element of the list to a list.
        return [list(x) for x in cur.fetchall()]

get_winners_by_name("Robert")

6. Create a function multiple_oscar_winners(num_oscars) that returns a dict of all oscar winners that has won "num_oscars" or more Oscars.

In [ ]:
def multiple_oscar_winners(num_oscars):
    oscar_winners = {}
    with conn.cursor() as cur:
        cur.execute("SELECT name, count(*) FROM oscar_winners HAVING count(*) >= %s", [num_oscars])
        for x in cur.fetchall():
            oscar_winners[x[0]] = x[1]
    return oscar_winners

7. Create a function write_oscar_winner(name, age, year, movie) that writes a new row to the database.

In [ ]:
def write_oscar_winner(year, age, name, movie):
    with conn.cursor() as cur:
        cur.execute("INSERT INTO oscar_winners (year, age, name, movie) VALUES (%s, %s, %s, %s);", [year, age, name, movie])
        conn.commit()

8. Download the file "female_oscar_winners.csv", but this time you are not allowed to use the DBeaver import function<br> to insert them to the database! Use the "csv" library together with the necessary functions you created in the tasks above.

In [ ]:
import csv

def clean_row(row):
    for k,v in row.items():
        if k is not None:
            row[k] = v.replace('"', '') # Remove all junk quotechars

def import_female_oscar_winners():
    with open("oscar_age_female.csv") as fp:
        reader = csv.DictReader(fp, fieldnames=["Index", "Year", "Age", "Name", "Movie"], dialect="excel", quotechar='"', delimiter=",")
        # Skip first row since it is the header
        next(reader)
        # Since this CSV file is a bit messy, itereate through each row and see if it has a column with none.
        # If it does, add the value of the "None" key to the movie.
        for row in reader:
            if row.get(None) is not None:
                row["Movie"] += "," + row[None][0]
        # Remove extra " characters from the columns
            clean_row(row)
            write_oscar_winner(row["Year"], row["Age"], row["Name"], row["Movie"])

import_female_oscar_winners()